In [1]:
import numpy as np

In [2]:
ground_truth_dataset = np.load("../dataset/random_5_p1_1_p5.npy")
ground_truth_dataset.shape

(200, 200, 2)

In [3]:
def make_dataset(data, history_len=5):
    """
    data: np.array of shape (N, T, 2)
    history_len: number of past points used for prediction
    Returns:
      X: np.array of shape (M, history_len, 2)
      y: np.array of shape (M, 2)
    """
    N, T, D = data.shape
    X_list, y_list = [], []
    for traj in data:                      # loop over N trajectories
        for t in range(T - history_len):  # t = 0,1,...,T-5-1
            hist = traj[t : t + history_len]   # shape (5,2)
            target = traj[t + history_len]     # shape (2,)
            X_list.append(hist)
            y_list.append(target)
    X = np.stack(X_list, axis=0)  # (M,5,2)
    y = np.stack(y_list, axis=0)  # (M,2)
    return X, y

In [4]:
X, y = make_dataset(ground_truth_dataset)
print(X.shape)
print(y.shape)

(39000, 5, 2)
(39000, 2)


In [5]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

M, H, D = X.shape
X_flat = X.copy().reshape(M, H*D)
X_train, X_val, y_train, y_val = train_test_split(
    X_flat, y,
    test_size=0.2,
)

model = Ridge(alpha=1.0)
model.fit(X_train, y_train)

print("R² (val):", model.score(X_val, y_val))

R² (val): 0.9999830617018373


In [6]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

def make_multi_step_preds_vec(X, model, history_len=5, tau=20, xy_limit=(0,5)):
    """
    X:        (N, T, 2)
    model:    fitted sklearn regressor mapping (history_len*2) → 2
    history_len: number of past frames
    tau:      how many steps to roll forward

    Returns
    -------
    preds:    (N, T, tau, 2)
    """
    N, T, D = X.shape
    assert D == 2

    pad_amount = history_len - 1
    pad = np.repeat(X[:, :1, :], pad_amount, axis=1)
    Xpad = np.concatenate([pad, X], axis=1)  # (N, T+hist_len-1, 2)

    windows = sliding_window_view(Xpad, window_shape=history_len, axis=1) # (N, T, 2, hist_len)
    windows = windows.transpose(0, 1, 3, 2)  # (N, T, hist_len, 2)

    preds = np.zeros((N, T, tau, 2), dtype=X.dtype)
    current = windows  # shape (N, T, hist_len, 2)

    for k in range(tau):
        feats = current.reshape(N*T, history_len*2)  # (N*T, hist_len*2)
        nxt = model.predict(feats) # (N*T, 2)
        nxt = nxt.reshape(N, T, 2) # (N, T, 2)
        preds[:, :, k, :] = nxt

        # row window forward
        current = np.concatenate([
            current[:, :, 1:, :], # drop lag-0
            nxt[:, :, None, :]    # append at end
        ], axis=2)

    return np.clip(preds, *xy_limit)

In [7]:
prediction_dataset = make_multi_step_preds_vec(ground_truth_dataset, model)
np.save("../dataset/random_5_p1_1_p5_pred_20", prediction_dataset)

In [8]:
import os, pickle

save_dir = "../dataset/"
os.makedirs(save_dir, exist_ok=True)

model_path = os.path.join(save_dir, "random_5_p1_1_p5_ridge_model.pkl")
with open(model_path, "wb") as f:
    pickle.dump(model, f)